<a href="https://colab.research.google.com/github/ajitonelsonn/TELE_Feedback/blob/main/Llama_Chatbot_with_Sentiment_Analysis_Integration_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TELE Feedback**

TELE Feedback is an AI-powered chatbot designed for a telecommunications company in Timor-Leste. It allows customers to provide feedback while integrating sentiment analysis to assess the tone of their messages. The project includes a simple interface for interaction and visualization of sentiment trends.

## 1.**Installing Required Libraries**

This cell installs all the libraries that the project depends on. These libraries are necessary for different functionalities like interacting with the Hugging Face models, creating a web interface for the chatbot, handling database operations, and visualizing feedback data.

- **`huggingface_hub`**: Allows the project to access pre-trained models from Hugging Face's model hub.
- **`gradio`**: Used to create an interactive web interface for the chatbot where users can input text and get responses.
- **`transformers`**: A library from Hugging Face that provides tools to use pre-trained models for natural language processing tasks, like language generation and sentiment analysis.
- **`sqlalchemy`**: An SQL toolkit for Python that helps in managing the storage and retrieval of data in a database.
- **`pandas`**: Used for data manipulation and analysis.
- **`matplotlib`**: A popular plotting library used for visualizing data, such as feedback trends.
- **`torch`**: PyTorch is a framework used for loading and running machine learning models.

In [1]:
!pip install huggingface_hub gradio git+https://github.com/huggingface/transformers sqlalchemy pandas matplotlib torch

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-51c7wg_i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-51c7wg_i
  Resolved https://github.com/huggingface/transformers to commit e15687fffe5c9d20598a19aeab721ae0a7580f8a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.2 MB/s eta

## 2.**Importing Necessary Libraries**

This cell imports all the necessary libraries and modules required throughout the project.

- **`huggingface_hub.login`**: Used to log in to Hugging Face, so you can access models stored in your account or publicly available models.
- **`gradio`**: Used for creating a user interface where people can interact with your chatbot.
- **`transformers.AutoModelForCausalLM` and `AutoTokenizer`**: These two classes load a pre-trained language model and its tokenizer, which are essential for generating chatbot responses.
- **SQLAlchemy-related imports**:
  - **`create_engine`**: Connects the code to the SQLite database.
  - **`Column`, `Integer`, `String`**: These are data types used to define the database schema.
  - **`declarative_base`**: A base class used to define the database table structure.
  - **`sessionmaker`**: Creates sessions to interact with the database.
- **`torch`**: PyTorch framework, required to load and run machine learning models like the chatbot.
- **`matplotlib.pyplot`**: This is used to plot graphs or charts, such as visualizing feedback data trends.
- **`collections.Counter`**: This utility helps in counting the occurrences of feedback sentiments (positive, negative, neutral).

In [2]:
from huggingface_hub import login
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import torch
import matplotlib.pyplot as plt
from collections import Counter

## 3.**Hugging Face Login**

This cell is responsible for logging into Hugging Face using your personal API token. If you don’t log in, you won’t be able to access models that are either private or require authentication.

- **`login(token="your_huggingface_token_here")`**: Replace `"your_huggingface_token_here"` with your actual token from Hugging Face. This allows the script to access models and datasets from the Hugging Face platform.

In [3]:
# Hugging Face login (you need to provide your token)
login(token="your_huggingface_token_here")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful



## 4.**Loading the Pre-Trained Model and Tokenizer**

This cell loads the language model and tokenizer that will be used to generate chatbot responses.

- **`AutoTokenizer.from_pretrained("model_name")`**: This loads a tokenizer from Hugging Face. The tokenizer processes raw text input and converts it into tokens that the model can understand.
- **`AutoModelForCausalLM.from_pretrained("model_name")`**: This loads the pre-trained language model. A causal language model (like GPT) is capable of generating text responses based on the input tokens.

The `model_name` should be replaced with the actual name of the pre-trained model you are using (for example, "gpt-3", "gpt-2", etc.). In this project we use **NousResearch/Llama-2-7b-chat-hf**

In [4]:
# Load Llama 2 model
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## 5.**Database Setup**

This code sets up a database for the project using SQLAlchemy's ORM.

- **`Base = declarative_base()`**: This creates a base class for all the declarative models.
- **`engine = create_engine('sqlite:///telecom_support.db', echo=False)`**: This creates a SQLite database named `telecom_support.db` and an engine to communicate with it.
  - **`echo=False`**: Suppresses SQLAlchemy log output.
- **`Session = sessionmaker(bind=engine)`**: Creates a session factory, which allows you to interact with the database.
- **`class Feedback(Base)`**: Defines a class `Feedback` that represents a table in the database.
  - **`__tablename__ = 'feedback'`**: The name of the table in the database.
  - **`id = Column(Integer, primary_key=True)`**: The primary key column for the feedback records.
  - **`category = Column(String)`**: Stores the category of the feedback.
  - **`original_text = Column(String)`**: Stores the original text of the feedback.
  - **`sentiment = Column(String)`**: Stores the sentiment of the feedback (e.g., positive or negative).
  - **`response = Column(String)`**: Stores the generated response for the feedback.
- **`Base.metadata.create_all(engine)`**: This line creates the tables in the database if they don't already exist.

In [5]:
# Database setup
Base = declarative_base()
engine = create_engine('sqlite:///telecom_support.db', echo=False)
Session = sessionmaker(bind=engine)

class Feedback(Base):
    __tablename__ = 'feedback'
    id = Column(Integer, primary_key=True)
    category = Column(String)
    original_text = Column(String)
    sentiment = Column(String)
    response = Column(String)

Base.metadata.create_all(engine)

<ipython-input-5-d23fa3de50b6>:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## 6.**Updated Categories for a Telecom Company**

This code defines a list of categories for a telecom company. Each category represents a specific area of service or product offering.

- **`Mobile Network`**: Refers to issues or feedback related to the telecom company's mobile network services, such as signal strength or coverage.
- **`Mobile Package`**: Deals with feedback about mobile plans or packages, such as data, call minutes, or SMS bundles.
- **`Mosan Emoney`**: Refers to feedback concerning the telecom's electronic money services or mobile wallet.
- **`Recharge`**: Encompasses issues or feedback related to recharging mobile credit or data balances.

In [6]:
# Updated categories for a telecom company
CATEGORIES = ["Mobile Network", "Mobile Package", "Mosan Emoney", "Recharge"]

## 7.**Response Generation Function**

This function generates a response based on the given prompt using a pre-trained language model.

- **`prompt`**: The input text or prompt for which a response is generated.
- **`max_length=500`**: Sets the maximum length of the generated response (default is 500 tokens).
- **`inputs = tokenizer(prompt, return_tensors="pt").to(model.device)`**:
  - **`tokenizer`**: Converts the prompt into tokenized tensor format.
  - **`return_tensors="pt"`**: Converts the input into PyTorch tensor format.
  - **`.to(model.device)`**: Moves the input to the device (CPU/GPU) where the model is loaded.
- **`outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)`**:
  - **`model.generate()`**: Generates the response based on the input tokens.
  - **`max_length`**: Limits the length of the generated output.
  - **`num_return_sequences=1`**: Returns one generated response.
  - **`temperature=0.7`**: Adjusts the randomness in the response generation. Lower values like 0.7 make responses more focused and deterministic.
- **`return tokenizer.decode(outputs[0], skip_special_tokens=True)`**: Decodes the generated tokens back into readable text, omitting special tokens.

In [7]:
def generate_response(prompt, max_length=500):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)

    # Decode the generated response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## 8.**Processing Customer Feedback**

This function processes customer feedback and generates a professional response based on the feedback's sentiment and content.

- **`message`**: The customer's feedback.
- **`category`**: The category of the feedback (e.g., Mobile Network, Mobile Package).
- **`prompt`**: The structured prompt provided to the model to generate a response.
- **`generate_response(prompt, max_length=500)`**: Generates a response using the AI model, with a maximum length of 500 tokens.
- **`return response`**: Returns the generated customer service response based on the feedback.

In [8]:
def process_feedback(message, category):
    prompt = f"""
    You are a professional customer service assistant for a telecom company. A customer has submitted the following feedback in the category of {category}:

    "{message}"

    Please respond with:
    1. A clear identification of the sentiment (positive, negative, or neutral).
    2. A concise, empathetic, and professional response addressing the customer's concern.
    3. If appropriate, include a solution or next steps for the customer.

    Format the output as:
    Sentiment: [positive/negative/neutral]
    Response: [Your customer response]

    No extra information or format explanations should be included.
    """

    # Generate response from the model
    response = generate_response(prompt, max_length=500)

    return response

## 9.**Chatbot with Error Handling and Feedback Storage**

This function processes customer feedback, identifies any errors, and stores the feedback with its sentiment in a database.

- **`message`**: The feedback text provided by the user.
- **`category`**: The category related to the feedback (e.g., Mobile Network, Mosan Emoney).
- **`process_feedback(message, category)`**: Processes the feedback based on its category using another function.
- **`generate_response(prompt, max_length)`**: Generates an AI response with a limit on the number of tokens.
- **`Session()`**: Creates a new session for database interaction.
- **`Feedback`**: A model class representing the feedback table in the database.
- **`session.add(feedback)`**: Adds the feedback to the database session.
- **`session.commit()`**: Saves the feedback to the database.
- **Error Handling**: If an error occurs, the bot generates an apology response using the AI model.

In [9]:
def chatbot(message, category):
    if not message.strip():
        return "Error: Please provide feedback text."
    if not category:
        return "Error: Please select a category."

    try:
        # Process the feedback with the category
        result = process_feedback(message, category)

        # Parse the result
        lines = result.split('\n')
        parsed_result = {}
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                parsed_result[key.strip()] = value.strip()

        # Extract sentiment and response
        sentiment = parsed_result.get('Sentiment', 'Unknown')
        response = parsed_result.get('Response', '')

        # If no response is generated, use a fallback
        if not response:
            fallback_prompt = f"Provide a brief, helpful response to a customer's feedback about {category}: '{message}'"
            response = generate_response(fallback_prompt, max_length=150)

        # Save the feedback to the database
        session = Session()
        feedback = Feedback(
            category=category,
            original_text=message,
            sentiment=sentiment,
            response=response
        )
        session.add(feedback)
        session.commit()

        return response

    except Exception as e:
        error_prompt = f"Briefly apologize and offer assistance for an error in processing feedback about {category}: '{message}'"
        error_response = generate_response(error_prompt, max_length=100)
        return error_response

## 10.**Gradio interface** for a telecommunication customer support chatbot.
This function is allows users to select a feedback category, enter feedback, and submit it to receive a chatbot-generated response.

1. **`gr.Blocks()`**: This defines the structure of the Gradio interface using block elements.
   
2. **`gr.Markdown("# Telecommunication Customer Support Chatbot")`**: Displays a title for the chatbot interface.

3. **`category = gr.Dropdown(...)`**: Creates a dropdown for the user to select the feedback category. The options are defined by the `CATEGORIES` list (not shown here).

4. **`message = gr.Textbox(...)`**: Provides a textbox for users to input their feedback. It allows for multi-line input (3 lines) and includes a placeholder text for guidance.

5. **`output = gr.TextArea(...)`**: A large text area where the chatbot's response will be displayed, with enough space (10 lines) to handle longer replies.

6. **`submit_btn = gr.Button("Submit Feedback")`**: Adds a button labeled "Submit Feedback" for users to send their input.

7. **`submit_btn.click(...)`**: Connects the button to the `chatbot` function. When clicked, it sends the feedback (`message`) and selected `category` as inputs, and displays the output in the `output` TextArea.

8. **`gr.Examples(...)`**: Displays preset feedback examples with categories and messages. Users can click these examples to autofill the input fields for demonstration purposes.



In [12]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Telecommunication Customer Support Chatbot")

    category = gr.Dropdown(choices=CATEGORIES, label="Select Feedback Category")
    message = gr.Textbox(lines=3, placeholder="Type your feedback here...", label="Your Feedback")

    # Use TextArea for better long-text handling
    output = gr.TextArea(label="Chatbot Response", lines=5)

    submit_btn = gr.Button("Submit Feedback")
    submit_btn.click(fn=chatbot, inputs=[message, category], outputs=output)

    gr.Examples(
        examples=[
            ["Mobile Network", "The network is down in my area for days."],
            ["Mobile Package", "I'm happy with the new data package!"],
            ["Mosan Emoney", "I have an issue with my last Mosan Emoney transfer."],
            ["Recharge", "I need help with my recent recharge transaction because already many times but not succsess."]
        ],
        inputs=[category, message]
    )

### **Start and display the Gradio interface.**

- **`demo.launch()`**: This method launches the Gradio app, making it interactive. Once called, it opens the interface in a browser or displays a link (if running in a cloud environment like Google Colab) that allows users to interact with the chatbot.

In [13]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ff82e60931adcdde78.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 11.**Visualizes customer sentiment trends**

This function plots the distribution of sentiments (positive, negative, neutral) across different feedback categories based on data from the database.

1. **Database Query**:
   - It queries the database to retrieve all the feedback using `session.query(Feedback).all()`.
   - The session is then closed to release the connection to the database.

2. **Data Processing**:
   - It uses a dictionary `category_sentiment_count` to store the count of each sentiment per feedback category.
   - Sentiment values are converted to lowercase to ensure consistency when comparing.

3. **Prepare Data for Plotting**:
   - `categories`: List of all feedback categories.
   - For each category, it counts the number of positive, negative, and neutral feedbacks.

4. **Plotting**:
   - The function uses `matplotlib` to create a bar plot with three sets of bars: one for each sentiment (positive, negative, neutral).
   - `bar_width` controls the width of each bar, and the x-positions of the bars are adjusted so they don't overlap.
   - It customizes the plot with labels for the x-axis (feedback categories), y-axis (number of feedback), and a title.

5. **Saving the Plot**:
   - The plot is saved as an image file named `sentiment_distribution.png` using `plt.savefig()`.
   - The plot is then closed to avoid display issues, especially in environments where multiple plots are generated.

6. **Return**:
   - The function returns the path to the saved image file, which can be displayed in a Gradio interface.


In [14]:
# Function to plot the data
def plot_sentiment_distribution():
    # Query the database to retrieve the feedback data
    session = Session()
    feedback_data = session.query(Feedback).all()

    # Processing data: count sentiments per category
    category_sentiment_count = {}

    for feedback in feedback_data:
        category = feedback.category
        sentiment = feedback.sentiment.lower()  # Convert sentiment to lowercase for consistency

        if category not in category_sentiment_count:
            category_sentiment_count[category] = Counter()

        category_sentiment_count[category][sentiment] += 1

    session.close()

    # Prepare data for plotting
    categories = list(category_sentiment_count.keys())
    positive_counts = [category_sentiment_count[cat].get('positive', 0) for cat in categories]
    negative_counts = [category_sentiment_count[cat].get('negative', 0) for cat in categories]
    neutral_counts = [category_sentiment_count[cat].get('neutral', 0) for cat in categories]

    # Plotting the data with adjusted bar width and position
    x = range(len(categories))  # x-coordinates
    bar_width = 0.2  # Adjust the bar width

    plt.figure(figsize=(10, 6))

    # Plotting positive, negative, and neutral sentiment bars for each category
    plt.bar([i - bar_width for i in x], positive_counts, width=bar_width, label='Positive', align='center')
    plt.bar(x, negative_counts, width=bar_width, label='Negative', align='center')
    plt.bar([i + bar_width for i in x], neutral_counts, width=bar_width, label='Neutral', align='center')

    # Customizing the plot
    plt.xlabel('Feedback Categories')
    plt.ylabel('Number of Feedback')
    plt.title('Sentiment Distribution by Feedback Categories')
    plt.xticks(x, categories, rotation=45)
    plt.legend()

    # Save the plot as an image
    plt.tight_layout()
    plt.savefig('sentiment_distribution.png')  # Save the plot as an image
    plt.close()  # Close the plot to avoid display issues

    # Return the image path for Gradio to display
    return 'sentiment_distribution.png'

### **Gradio Interface for customer sentiment trends**
Interface will display the sentiment distribution plot for telecom customer feedback data without requiring any user input.

1. **`interface = gr.Interface(...)`**: This creates a Gradio interface for displaying the sentiment distribution plot generated by the `plot_sentiment_distribution` function.

2. **`fn=plot_sentiment_distribution`**: Specifies that the `plot_sentiment_distribution` function is the core function executed when the interface is run.

3. **`inputs=[]`**: No inputs are required from the user since the function retrieves and processes the data directly from the database.

4. **`outputs="image"`**: The output of the function is an image (the saved plot) which will be displayed in the interface.

5. **`title="Telecom Support Feedback Sentiment"`**: This is the title that appears at the top of the Gradio interface.



In [15]:
# Gradio interface
interface = gr.Interface(fn=plot_sentiment_distribution, inputs=[], outputs="image", title="Telecom Support Feedback Sentiment")

###Start and display the Gradio interface.

- **`interface.launch()`**: This starts the Gradio application, opening a browser window or providing a link to the interface if it's running in a cloud environment. The interface will display the sentiment distribution plot image generated by the `plot_sentiment_distribution` function.

In [16]:
# Launch the interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9467f561339b2a9cf7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


---

## **About the Author**

**Ajito Nelson** is a passionate **Big Data Engineer** and **AI Enthusiast**. With a deep interest in data-driven solutions and artificial intelligence, Ajito is dedicated to leveraging cutting-edge technology to solve complex challenges. His expertise spans across big data infrastructure, machine learning, and AI applications in various domains.

You can connect with me on [LinkedIn](https://www.linkedin.com/in/ajitonelson/).

![Aji To Nelson](https://media.licdn.com/dms/image/v2/C5603AQENYxgypX_VSg/profile-displayphoto-shrink_800_800/profile-displayphoto-shrink_800_800/0/1659079610353?e=1732752000&v=beta&t=H5SDXNRWwwIviJ1lP8muSj9Xb2Aa0rCklbaZ0Hgjnf8)